In [1]:
%pip install pandas
%pip install dask

import gc
import pandas as pd
import dask.dataframe as dd

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


/Users/ana/Documents/Ocean/DataSharing/myenv/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
input_parquet_file_path = '../data/interm/2.0_taxdata.parquet'
output_parquet_file_path = '../data/interm/2.1_taxdata.parquet'

In [3]:
ddf = dd.read_parquet(input_parquet_file_path)

# Correct the code INSEE field

In [4]:
# Define the function to correct and validate INSEE codes
def correct_insee_code(commune, department):
    formatted_dept_code = department

    try:
        if len(department) == 1:
            formatted_dept_code = "0" + department
        
        if len(department) == 3:
            commune_code = f"{int(commune):02d}"
            
        else:
            commune_code = f"{int(commune):03d}"

    except ValueError:
        commune_code = ''

    return f"{formatted_dept_code}{commune_code}"

def apply_corrections(row):
    if pd.isna(row['COMMUNE']) or row['COMMUNE'] == '' or pd.isna(row['DEPARTEMENT']) or row['DEPARTEMENT'] == '':
        return row['code INSEE']
    
    valid_code = correct_insee_code(row['COMMUNE'], row['DEPARTEMENT'])
    return valid_code

ddf['Clean code INSEE'] = ddf.map_partitions(lambda df: df.apply(apply_corrections, axis=1), meta=pd.Series(dtype='str'))

# Compute the result
result_df = ddf.compute()

In [5]:
# Save the processed data back to a Parquet file
result_df.to_parquet(output_parquet_file_path)

num_unique_clean_code_insee = result_df['Clean code INSEE'].nunique()
print(f'Number of communes: {num_unique_clean_code_insee}')

del ddf
del result_df
gc.collect()

Number of communes: 36760


0